<a href="https://colab.research.google.com/github/RoshanTS/pfe-estia/blob/extract_twitter_script/extraction_donn%C3%A9es_via_api_twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Script d'extraction de données twitter

##Importations of Python modules

We import the following libraries :

*   [tweepy](https://docs.tweepy.org/en/latest/) :  python client for the official [Twitter API](https://developer.twitter.com/en.html) 
*   [textblob](https://docs.tweepy.org/en/latest/) : python library for processing textual data
*  [os](https://docs.python.org/fr/2.7/library/os.html)
 : contient diverses fonctions pour gérer l'os

* [pandas](https://pandas.pydata.org/pandas-docs/version/0.25/) : python library for data analysis

* [re](https://docs.python.org/3/library/re.html) : python library for regular expressions

* [nltk](https://www.nltk.org/) : python library to do things like   tokenization, stemming, ..

In [0]:
import tweepy 
import textblob
import os
import pandas
import re
import nltk


We're going to use [OAuthHandler](http://docs.tweepy.org/en/v3.5.0/auth_tutorial.html) to deal with authentification :


In [0]:
from tweepy import OAuthHandler 
from textblob import TextBlob 

## Quick sentiment analysis

*We* define a classe Twitter client :

In [0]:
class TwitterClient(object): 
	''' 
	Generic Twitter Class for sentiment analysis. 
	'''
	def __init__(self): 
		''' 
		Class constructor or initialization method. 
		'''
		# keys and tokens from the Twitter Dev Console 
		consumer_key = 'MlbpfXnAwJeQzQlRYK0dKJ6nB'
		consumer_secret = 'cTZLfoM7xOHpFjSfzUW0Ff0AsKPNsqEOhjMNiMbNW6Eg0NZpWr'
		access_token = '1179886074614689793-VLHbcHpSJoXkml9wVMqd9gIcSs4zMS'
		access_token_secret = 'Vnpf7cZp2uIUsdDDV6egSHdxDQXyCHf7INdKVa8Uqs2H4'

		# attempt authentication 
		try: 
			# create OAuthHandler object 
			self.auth = OAuthHandler(consumer_key, consumer_secret) 
			# set access token and secret 
			self.auth.set_access_token(access_token, access_token_secret) 
			# create tweepy API object to fetch tweets 
			self.api = tweepy.API(self.auth) 
		except: 
			print("Error: Authentication Failed") 

	def clean_tweet(self, tweet): 
		''' 
		Utility function to clean tweet text by removing links, special characters 
		using simple regex statements. 
		'''
		return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) 

	def get_tweet_sentiment(self, tweet): 
		''' 
		Utility function to classify sentiment of passed tweet 
		using textblob's sentiment method 
		'''
		# create TextBlob object of passed tweet text 
		analysis = TextBlob(self.clean_tweet(tweet)) 
		# set sentiment 
		if analysis.sentiment.polarity > 0: 
			return 'positive'
		elif analysis.sentiment.polarity == 0: 
			return 'neutral'
		else: 
			return 'negative'

	def get_tweets(self, query, count = 10): 
		''' 
		Main function to fetch tweets and parse them. 
		'''
		# empty list to store parsed tweets 
		tweets = [] 

		try: 
			# call twitter api to fetch tweets 
			fetched_tweets = self.api.search(q = query, count = count) 

			# parsing tweets one by one 
			for tweet in fetched_tweets: 
				# empty dictionary to store required params of a tweet 
				parsed_tweet = {} 

				# saving text of tweet 
				parsed_tweet['text'] = tweet.text 
				# saving sentiment of tweet 
				parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text) 

				# appending parsed tweet to tweets list 
				if tweet.retweet_count > 0: 
					# if tweet has retweets, ensure that it is appended only once 
					if parsed_tweet not in tweets: 
						tweets.append(parsed_tweet) 
				else: 
					tweets.append(parsed_tweet) 

			# return parsed tweets 
			return tweets 

		except tweepy.TweepError as e: 
			# print error (if any) 
			print("Error : " + str(e)) 


We define a function main :


1.   query sur l'api twitter
2.   print positive versus negative sentiment analysis



In [0]:

def main(query): 
	# creating object of TwitterClient Class 
	api = TwitterClient() 
	# calling function to get tweets 
	tweets = api.get_tweets(query, count = 200) 

	# picking positive tweets from tweets 
	ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive'] 
	# percentage of positive tweets 
	print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets))) 
	# picking negative tweets from tweets 
	ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative'] 
	# percentage of negative tweets 
	print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets))) 
	# percentage of neutral tweets 
#	print("Neutral tweets percentage: {} %".format(100*len(tweets - ntweets - ptweets)/len(tweets))) 

	# printing first 5 positive tweets 
	print("\n\nPositive tweets:") 
	for tweet in ptweets[:10]: 
		print(tweet['text']) 

	# printing first 5 negative tweets 
	print("\n\nNegative tweets:") 
	for tweet in ntweets[:10]: 
		print(tweet['text']) 


We apply our analysis to the tweets featuring "Hong Kong":

In [0]:

if __name__ == "__main__": 
	# calling main function 
	main('#HongKong') 


Positive tweets percentage: 23.728813559322035 %
Negative tweets percentage: 18.64406779661017 %


Positive tweets:
RT @joshuawongcf: Last week, the #HongKong Government invoked the Emergency Regulations Ordinance to outlaw face masks, curbing our right t…
RT @Jay_Watt: Harry Harrison #cartoon in today's @SCMPNews pointing out the new normal for the #HongKong police: they absolutely will deman…
RT @realKyleOlbert: I met a kind old man from #Guangdong Province today &amp; asked him for his opinion on #HongKong.

I was startled to hear h…
RT @9GAG: Blizzard: Not us
#HongKong 
-
Original comic: @JakeClarkDude https://t.co/Kp9ACaz4c1
RT @BoycottHegemony: This is the #CCP world we live in now. WAKE UP!

A Sydney #gamingstartup has been hit by a concerted #cyberattack afte…
RT @SenRickScott: As the first Senator to visit #HongKong since the protests started, my message is simple. To the brave people of HK: We a…
RT @patrickyflee: Female student of the Chinese University of Hong Kong removes 

## Extract twitter data from a query

### Authentifaction parameters

In [0]:
dict_key_token = {'consumer_key' : 'MlbpfXnAwJeQzQlRYK0dKJ6nB',
                  'consumer_secret' : 'cTZLfoM7xOHpFjSfzUW0Ff0AsKPNsqEOhjMNiMbNW6Eg0NZpWr',
                  'access_token' : '1179886074614689793-VLHbcHpSJoXkml9wVMqd9gIcSs4zMS',
                  'access_token_secret' : 'Vnpf7cZp2uIUsdDDV6egSHdxDQXyCHf7INdKVa8Uqs2H4'
                 }


auth = tweepy.OAuthHandler(dict_key_token['consumer_key'], dict_key_token['consumer_secret'])
auth.set_access_token(dict_key_token['access_token'], dict_key_token['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

### Part 1 : how to search for tweets ?

We define variables linked to our search :

In [0]:
#content of our query
search_words = "Hong Kong"

#we want tweets starting from ..
date_since = "2019-10-01"

#we want a given number of tweets
nb_tweets = 10

Then , we collect tweets (note that we could specify the language) and print them :

In [0]:
#Collect tweets
tweets = tweepy.Cursor(api.search,
                      q=search_words,
                      lang="en",  
                       since=date_since).items(nb_tweets)

# Iterate and print tweets

for tweet in tweets:
    print(tweet.text)
  

To be honest,
the whole #BoycottBlizzard seems like hindsight.
I mean, what's the real reason that meddling with th… https://t.co/m8wszLAH4q
Good 3 Qs: “When was @hkmaplive “used maliciously to target individual officers for violence”?
When was it used to… https://t.co/VtRybpwAbv
RT @JesseKellyDC: REMINDER: There is no “impeachment probe”. The House hasn’t voted on it cause they don’t have the votes and their story i…
I think its so cool my great grandfather is from Hong Kong 🇭🇰
RT @Nicole__1022: A follow up: CCP netizens reaction to Adam Silvers statement on NBA

If I m not mistaken, thats a blatant statement calli…
RT @blakersdozen: Scoop: Blizzard employees walked out of work yesterday over the company's decision to ban a pro-Hong-Kong professional He…
RT @SpeakeasyJames: Bernier on the repatriation for 2 Canadians held in China.
Trudeau claims to defend Canadians (but after 2 years)
Schee…
RT @HawleyMO: Apple assured me last week that their initial decision to ban this app was a mi

### Part 2 : How to keep or remove retweets ?

In [0]:
new_search = search_words + " -filter:retweets"
new_search

Same as before, we collect :

In [0]:
#Collect tweets
tweets = tweepy.Cursor(api.search,
                      q=search_words,
                      lang="en",  
                       since=date_since).items(nb_tweets)

# Iterate and print tweets

[tweet.text for tweet in tweets]


["To be honest,\nthe whole #BoycottBlizzard seems like hindsight.\nI mean, what's the real reason that meddling with th… https://t.co/m8wszLAH4q",
 'Good 3 Qs: “When was @hkmaplive “used maliciously to target individual officers for violence”?\nWhen was it used to… https://t.co/VtRybpwAbv',
 'RT @JesseKellyDC: REMINDER: There is no “impeachment probe”. The House hasn’t voted on it cause they don’t have the votes and their story i…',
 'I think its so cool my great grandfather is from Hong Kong 🇭🇰',
 'RT @Nicole__1022: A follow up: CCP netizens reaction to Adam Silvers statement on NBA\n\nIf I m not mistaken, thats a blatant statement calli…',
 "RT @blakersdozen: Scoop: Blizzard employees walked out of work yesterday over the company's decision to ban a pro-Hong-Kong professional He…",
 'RT @SpeakeasyJames: Bernier on the repatriation for 2 Canadians held in China.\nTrudeau claims to defend Canadians (but after 2 years)\nSchee…',
 'RT @HawleyMO: Apple assured me last week that their init

### Part 3 : Who's tweeting about our topic ?

In [0]:
# Collect tweets
tweets = tweepy.Cursor(api.search, 
                           q=new_search,
                           lang="en",
                           since=date_since).items(5)



#Storing in a list and printing the collected tweets 
  #tweet.user.screen_name to get user's name
  #tweet.user.location to get user's provided location

users_locs = [[tweet.user.screen_name, tweet.user.location] for tweet in tweets]
users_locs

[['IPDefenseForum',
  'Blizzard won’t ban American players after Hong Kong protest, but team plans to forfeit over boycott #IndoAsiaPac… https://t.co/qzdgoUGX2t'],
 ['Bariter5', '@Blizzard_Ent Stand up for #HongKong. Revolution in our time'],
 ['GrownDirtStorm',
  '#HongKong #Blizzard #BoycottBlizzard I make me em https://t.co/dQHi2iPy8W'],
 ['nazbolqueen',
  'Blizzard with some Lannister tier punishments on gamers for discussing Hong Kong. \n\n#HongKongProtest #HongKong… https://t.co/bajO92LutJ'],
 ['LillyMuse4',
  'From @reddit \n.\n.\n.\n.\n.\n.\n.\n.\n#blizzard #blizzardentertainment #blizzardgames #hearthstone #overwatch… https://t.co/Vw8JzluSif']]

### Part 3 : Create a pandas dataframe from a list of tweets

In [0]:
tweet_text = pandas.DataFrame(data=users_locs, 
                    columns=['user', "location"])
tweet_text

,user,location
0,nigelcameron,"Chiefly Brussels, but also London, DC, Chicago"
1,transgrrl,
2,Hoodlum420,"Hollywood, CA"
3,MichaelDavSmith,Chicago
4,GingeKingYT,


We have empty rows for column 'location cause some users have refused to share their locations with Twitter.

### Part 5 : Customizing twitter queries

Look at [twitter API](https://developer.twitter.com/en/docs/tweets/rules-and-filtering/overview/standard-operators) for more infos.

We're going to make a query with "Hongkong + blizzard" and collect the tweets related : 

In [0]:
new_search = "hongkong  -filter:retweets"

tweets = tweepy.Cursor(api.search,
                   q=new_search,
                   #lang="en",
                   since='2018-04-23').items(2000)

#all_tweets = [tweet.text for tweet in tweets]
#all_tweets[:5]


We get tweet informations and store everything in a dataframe : 

In [100]:

users_infos = [[tweet.user.screen_name, tweet.user.location, tweet.text,tweet.lang,tweet.favorite_count,tweet.retweet_count,tweet.created_at] for tweet in tweets]
users_infos



df_tweets = pandas.DataFrame(data=users_infos, 
                    columns=['user', "location","tweet content","language","nb_favorite","nb_retweet","created_at"])
df_tweets



,user,location,tweet content,language,nb_favorite,nb_retweet,created_at
0,Hei0v0,,@SenAngusKing @marcorubio @senatemajldr Thank ...,en,0,0,2019-10-18 00:36:47
1,WinChan05999504,,@SenAngusKing @pinkykiki222 @marcorubio @senat...,en,0,0,2019-10-18 00:36:44
2,muimui_hongkong,,香港帰りてぃ😭,ja,0,0,2019-10-18 00:36:16
3,lia286848909,,@ajmm19923493 Never forgot. Never forgive.\n#F...,en,0,0,2019-10-18 00:36:15
4,canativeobt,OCEAN BEACH SD CALIF,Hong Kong assembly in chaos; attack on democra...,en,0,0,2019-10-18 00:35:55
5,SiddLively,,@NBAStatGuy_ @The_Alex_Barber @clintisiceman @...,en,0,0,2019-10-18 00:35:44
6,aprilwg,Hong Kong,Real #HongKongers won’t attack ethical minorit...,en,0,0,2019-10-18 00:35:20
7,k0kOT9lIijVx3GF,,＃hongkong 一群话都答不上来的人，口口声声要救香港 https://t.co/w3Z...,zh,1,1,2019-10-18 00:35:11
8,Zzchuja,Hong Kong,@SCMPNews I think it is the 4th time that Rese...,en,0,0,2019-10-18 00:34:51
9,SPICCA0911,,@RepZoeLofgren Thankyou very much for supporti...,en,0,0,2019-10-18 00:34:32


##Export a csv to drive or on local

In [0]:
# to export from google collab :
from google.colab import files

#add datetime to filename
import time
date = time.strftime("%d-%m-%Y__%H:%M:%S")
savename = "df_tweets_"+date+".csv" 



### sur le drive 

Follow these steps to do save on the drive :


1. We need to setup our drive first, run this :


In [187]:
#need to do this only once
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).



2.   Click on the URL, choose which account you're workin on, and copy paste the link in the box. You should get something like this :
![](https://miro.medium.com/max/2234/1*ZpgiKaB2RSo_nTAvo9QNbg.png)


3. [optionnal] You could naviguate to your folder and see what you have in your drive, here we want to go there. Useful when you want to define path variable :



In [188]:
!ls /content/gdrive/Shared\ drives/PFE_ING3_ESTIA/data

df_tweets_18-10-2019.csv  df_tweets.csv  df_tweets_rouen_fire.csv


4. Setup done! We need to save our model in GDrive :

In [210]:
print("savename : ",savename,"\n","path : ",path)

savename :  df_tweets_18-10-2019__02:23:32.csv 
 path :  /content/gdrive/Shared drives/


In [0]:
df_tweets.to_csv(savename)

# copy paste savename and path printed below
#!cp savename path

#work but nor adaptative (ex : CRON ...) 
!cp df_tweets_18-10-2019__02:23:32.csv /content/gdrive/Shared\ drives/PFE_ING3_ESTIA/data

#good for cron extract but need to work on shared drives
# import shutil
# path = "/content/gdrive/Shared drives/"
# shutil.copy(savename, path)


In [215]:
!ls /content/gdrive/Shared\ drives/PFE_ING3_ESTIA/data

df_tweets_18-10-2019__02:23:32.csv  df_tweets.csv
df_tweets_18-10-2019.csv	    df_tweets_rouen_fire.csv


In [199]:
!pwd

/content


### en local




In [0]:
df_tweets.to_csv(savename)
files.download(savename)